In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **INSTALL PACKAGE**

In [ ]:
!pip install tensorflow-io
!pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 5.6 MB/s eta 0:00:00


In [ ]:
!pip install vncorenlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645933 sha256=b56c28bb6c5512d073d9f0c2c01eee323bc597bca82c85252199e15ab7c92091
  Stored in directory: /root/.cache/pip/wheels/5d/d9/b3/41f6c6b1ab758561fd4aab55dc0480b9d7a131c6aaa573a3fa
Successfully built vncorenlp


In [ ]:
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

--2023-07-21 05:46:08--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M   126MB/s    in 0.2s    

2023-07-21 05:46:09 (126 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [27412575/27412575]

--2023-07-21 05:46:09--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting respon

In [ ]:
!pip install emot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 1.4 MB/s eta 0:00:00


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.0 MB/s eta 0:00:00


# **IMPORT LIBRARY**

In [ ]:
import os
import re
import sys
import emot
import time
import json
import requests
import threading
import traceback
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm
import dateutil.parser
import tensorflow as tf
from random import randint
import tensorflow_io as tfio
from datetime import datetime
from vncorenlp import VnCoreNLP
from kafka import KafkaProducer
from kafka.errors import KafkaError
from sklearn.model_selection import train_test_split
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO
from transformers import XLMRobertaTokenizer, XLMRobertaModel

In [ ]:
print("tensorflow-io version: {}".format(tfio.__version__))
print("tensorflow version: {}".format(tf.__version__))

tensorflow-io version: 0.32.0
tensorflow version: 2.12.0


# **SETUP KAFKA**
Download and setup kafka zookeeper and kafka server

In [ ]:
!curl -sSOL https://downloads.apache.org/kafka/3.4.1/kafka_2.13-3.4.1.tgz
!tar -xzf kafka_2.13-3.4.1.tgz

In [ ]:
!./kafka_2.13-3.4.1/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.4.1/config/zookeeper.properties
!./kafka_2.13-3.4.1/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.4.1/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


In [ ]:
!ps -ef | grep kafka

root        1480       1 15 05:47 ?        00:00:01 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:MaxGCPauseMillis=20 -XX:InitiatingHeapOccupancyPercent=35 -XX:+ExplicitGCInvokesConcurrent -XX:MaxInlineLevel=15 -Djava.awt.headless=true -Xlog:gc*:file=/content/kafka_2.13-3.4.1/bin/../logs/zookeeper-gc.log:time,tags:filecount=10,filesize=100M -Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Dkafka.logs.dir=/content/kafka_2.13-3.4.1/bin/../logs -Dlog4j.configuration=file:./kafka_2.13-3.4.1/bin/../config/log4j.properties -cp /content/kafka_2.13-3.4.1/bin/../libs/activation-1.1.1.jar:/content/kafka_2.13-3.4.1/bin/../libs/aopalliance-repackaged-2.6.1.jar:/content/kafka_2.13-3.4.1/bin/../libs/argparse4j-0.7.0.jar:/content/kafka_2.13-3.4.1/bin/../libs/audience-annotations-0.13.0.jar:/content/kafka_2.13-3.4.1/bin/../libs/commons-cli-1.4.jar:/content/kafka_2.13-3.4.1/bin/../libs/commons-lang3-3.8.1.jar:/content/kafka_2.1

# **PREPROCESS DATA**

In [ ]:
vncorenlp = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

In [ ]:
print(UNICODE_EMOJI)
print(EMOTICONS_EMO)

{'🥇': ':1st_place_medal:', '🥈': ':2nd_place_medal:', '🥉': ':3rd_place_medal:', '🆎': ':AB_button_(blood_type):', '🏧': ':ATM_sign:', '🅰': ':A_button_(blood_type):', '🇦🇫': ':Afghanistan:', '🇦🇱': ':Albania:', '🇩🇿': ':Algeria:', '🇦🇸': ':American_Samoa:', '🇦🇩': ':Andorra:', '🇦🇴': ':Angola:', '🇦🇮': ':Anguilla:', '🇦🇶': ':Antarctica:', '🇦🇬': ':Antigua_&_Barbuda:', '♒': ':Aquarius:', '🇦🇷': ':Argentina:', '♈': ':Aries:', '🇦🇲': ':Armenia:', '🇦🇼': ':Aruba:', '🇦🇨': ':Ascension_Island:', '🇦🇺': ':Australia:', '🇦🇹': ':Austria:', '🇦🇿': ':Azerbaijan:', '🔙': ':BACK_arrow:', '🅱': ':B_button_(blood_type):', '🇧🇸': ':Bahamas:', '🇧🇭': ':Bahrain:', '🇧🇩': ':Bangladesh:', '🇧🇧': ':Barbados:', '🇧🇾': ':Belarus:', '🇧🇪': ':Belgium:', '🇧🇿': ':Belize:', '🇧🇯': ':Benin:', '🇧🇲': ':Bermuda:', '🇧🇹': ':Bhutan:', '🇧🇴': ':Bolivia:', '🇧🇦': ':Bosnia_&_Herzegovina:', '🇧🇼': ':Botswana:', '🇧🇻': ':Bouvet_Island:', '🇧🇷': ':Brazil:', '🇮🇴': ':British_Indian_Ocean_Territory:', '🇻🇬': ':British_Virgin_Islands:', '🇧🇳': ':Brunei:', '🇧🇬': ':B

In [ ]:
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

In [ ]:
def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split('|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split('|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()

In [ ]:
def covert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

In [ ]:
def call_api(data, url, method, timeout=3):
    headers = {
        'content-type': "application/x-www-form-urlencoded",
        'cache-control': "no-cache",
        'postman-token': "6a410524-a8e2-79c7-bd9d-53e4b68c84c7"
    }
    response = requests.request(method, url, data=data, headers=headers, timeout=timeout)
    return response

In [ ]:
def convertwindown1525toutf8(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

In [ ]:
bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

In [ ]:
nguyen_am_to_ids = {}
for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

In [ ]:
def vn_word_to_telex_type(word):
    dau_cau = 0
    new_word = ''
    for char in word:
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            new_word += char
            continue
        if y != 0:
            dau_cau = y
        new_word += bang_nguyen_am[x][-1]
    new_word += bang_ky_tu_dau[dau_cau]
    return new_word

In [ ]:
def vn_sentence_to_telex_type(sentence):
    words = sentence.split()
    for index, word in enumerate(words):
        words[index] = vn_word_to_telex_type(word)
    return ' '.join(words)

In [ ]:
def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word
    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
        else:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    return ''.join(chars)

In [ ]:
def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True

In [ ]:
def chuan_hoa_dau_cau_tieng_viet(sentence):
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        words[index] = chuan_hoa_dau_tu_tieng_viet(word)
    return ' '.join(words)

In [ ]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'', text)

In [ ]:
def chuan_hoa_teen_code(sentence):
    code = ['ctrai', 'khôg', 'bme', 'cta', 'mih', 'mqh', 'cgai', 'nhữg', 'mng', 'svtn', 'r', 'qtam', 'thươg', 'qtâm', 'chug', 'trườg', 'thoy', 'đki', 'atsm', 'ạk', 'cv', 'vch', 'cùg', 'pn', 'pjt', 'thjk', 'keke', 'ktra', 'nek', 'cgái', 'nthe', 'chúg', 'kái', 'tìh', 'phòg', 'lòg', 'từg', 'rằg', 'sốg', 'thuj', 'thuơng', 'càg', 'đky', 'bằg', 'sviên', 'ák', 'đág', 'nvay', 'nhjeu', 'xg', 'zồi', 'trag', 'zữ', 'atrai', 'kte', 'độg', 'lmht', 'gắg', 'đzai', 'thgian', 'plz', 'đồg', 'btrai', 'nthê', 'hìhì', 'vọg', 'hihe', 'đôg', 'răg', 'thườg', 'tcảm', 'đứg', 'ksao', 'dz', 'hjxhjx', 'cmày', 'xuốg', 'nkư', 'lquan', 'tiếg', 'hajz', 'xih', 'hìh', 'thàh', 'ngke', 'dzậy', 'teencode', 'tnào', 'tưởg', 'ctrinh', 'phog', 'hôg', 'zìa', 'kũg', 'ntnao', 'trọg', 'nthế', 'năg', 'ngđó', 'lquen', 'riêg', 'ngag', 'hêhê', 'bnhiu', 'ngốk', 'kậu', 'highland', 'kqua', 'htrc', 'địh', 'gđình', 'giốg', 'csống', 'xug', 'zùi', 'bnhiêu', 'cbị', 'kòn', 'buôg', 'csong', 'chàg', 'chăg', 'ngàh', 'llac', 'nkưng', 'nắg', 'tíh', 'khoảg', 'thík', 'ngđo', 'ngkhác', 'thẳg', 'kảm', 'dàh', 'júp', 'lặg', 'vđê', 'bbè', 'bóg', 'dky', 'dòg', 'uốg', 'tyêu', 'snvv', 'đthoại', 'qhe', 'cviec', 'tượg', 'qà', 'thjc', 'nhưq', 'cđời', 'bthường', 'zà', 'đáh', 'xloi', 'zám', 'qtrọng', 'bìh', 'lzi', 'qhệ', 'đhbkhn', 'hajzz', 'kủa', 'lz', 'đhkhtn', 'đóg', 'cka', 'lgi', 'nvậy', 'qả', 'đkiện', 'nèk', 'tlai', 'bsĩ', 'hkì', 'đcsvn', 'vde', 'chta', 'òy', 'ltinh', 'ngyeu', 'đthoai', 'snghĩ', 'nặg', 'họk', 'dừg', 'hphúc', 'hiha', 'wtâm', 'thíck', 'chuện', 'lạh', 'fây', 'ntnày', 'lúk', 'haj', 'ngía', 'mớj', 'hsơ', 'ctraj', 'nyêu', 'điiiiiii', 'rồii', 'c', 'kih', 'kb', 'hixxx', 'dthương', 'nhiềuuu', 'ctrình', 'mìnk', 'mjh', 'ng', 'vc', 'uhm', 'thỳ', 'nyc', 'tks', 'nàg', 'thôii', 'đjên', 'bgái', 'vớii', 'xink', 'hđộng', 'đhọc', 'mk', 'bn', 'thik', 'cj', 'mn', 'nguoi', 'nógn', 'hok', 'ko', 'bik', 'vs', 'cx', 'mik', 'wtf', 'đc', 'cmt', 'ck', 'chk', 'ngta', 'gđ', 'oh', 'vk', 'ctác', 'sg', 'ae', 'ah', 'ạh', 'rì', 'ms', 'vn', 'nhaa', 'cũg', 'đag', 'ơiii', 'hic', 'ace', 'àk', 'uh', 'cmm', 'cmnr', 'ơiiii', 'hnay', 'ukm', 'tq', 'ctr', 'đii', 'nch', 'trieu', 'hahah', 'nta', 'ngèo', 'kêh', 'ak', 'ad', 'j', 'ny', 'dc', 'qc', 'baoh', 'zui', 'zẻ', 'tym', 'aye', 'eya', 'fb', 'insta', 'z', 'thich', 'vcl', 'đt', 'acc', 'lol', 'loz', 'lozz', 'trc', 'chs', 'đhs', 'qá', 'ntn', 'wá', 'zậy', 'zô', 'ytb', 'vđ', 'vchg', 'sml', 'xl', 'cmn', 'face', 'hjhj', 'vv', 'ns', 'iu', 'vcđ', 'in4', 'qq', 'sub', 'kh', 'zạ', 'oy', 'jo', 'clmm', 'bsvv', 'troai', 'wa', 'hjx', 'e', 'ik', 'ji', 'ce', 'lm', 'đz', 'sr', 'ib', 'hoy', 'đbh', 'k', 'vd', 'a', 'cũng z', 'z là', 'unf', 'my fen', 'fen', 'cty', 'on lai', 'u hai ba', 'kô', 'đtqg', 'hqua', 'xog', 'uh', 'uk', 'nhoé', 'biet', 'quí', 'stk', 'hong kong', 'đươc', 'nghành', 'nvqs', 'ngừoi', 'trog', 'tgian', 'biêt', 'fải', 'nguời', 'tđn', 'bth', 'vcđ', 'tgdd', 'khg', 'nhưg', 'thpt', 'thằg', 'đuợc', 'dc', 'đc', 'ah', 'àh', 'ku', 'thým', 'onl', 'zô', 'zú', 'cmnd', 'sđt', 'klq']
    chuanhoa = ['con trai', 'không', 'bố mẹ', 'chúng ta', 'mình', 'mối quan hệ', 'con gái', 'những', 'mọi người', 'sinh viên tình nguyện', 'rồi', 'quan tâm', 'thương', 'quan tâm', 'chung', 'trường', 'thôi', 'đăng ký', 'ảo tưởng sức mạnh', 'ạ', 'công việc', 'vãi chưởng', 'cùng', 'bạn', 'biết', 'thích', 'ce ce', 'kiểm tra', 'nè', 'con gái', 'như thế', 'chúng', 'cái', 'tình', 'phòng', 'lòng', 'từng', 'rằng', 'sống', 'thôi', 'thương', 'càng', 'đăng ký', 'bằng', 'sinh viên', 'á', 'đáng', 'như vậy', 'nhiều', 'xuống', 'rồi', 'trang', 'dữ', 'anh trai', 'kinh tế', 'động', 'liên minh huyền thoại', 'gắng', 'đẹp trai', 'thời gian', 'pờ ly', 'đồng', 'bạn trai', 'như thế', 'hì hì', 'vọng', 'hi he', 'đông', 'răng', 'thường', 'tình cảm', 'đứng', 'không sao', 'đẹp trai', 'hix hix', 'chúng mày', 'xuống', 'như', 'liên quan', 'tiếng', 'hai', 'xinh', 'hình', 'thành', 'nghe', 'dậy', 'tin cốt', 'thế nào', 'tưởng', 'chương trình', 'phong', 'không', 'gì', 'cũng', 'như thế nào', 'trọng', 'như thế', 'năng', 'người đó', 'làm quen', 'riêng', 'ngang', 'hê hê', 'bao nhiêu', 'ngốc', 'cậu', 'hai lừn', 'kết quả', 'hôm trước', 'định', 'gia đinh', 'giống', 'cuộc sống', 'xùng', 'rồi', 'bao nhiêu', 'chuẩn bị', 'còn', 'buông', 'cuộc sống', 'chàng', 'chăng', 'ngành', 'liên lạc', 'nhưng', 'nắng', 'tính', 'khoảng', 'thích', 'người đó', 'người khác', 'thẳng', 'cảm', 'dành', 'giúp', 'lặng', 'vấn đề', 'bạn bè', 'bóng', 'đăng ký', 'dòng', 'uống', 'tình yêu', 'sinh nhật vui vẻ', 'điện thoại', 'quan hệ', 'công việc', 'tượng', 'quà', 'thích', 'nhưng', 'cuộc đời', 'bình thường', 'già', 'đánh', 'xin lỗi', 'dám', 'quan trọng', 'bình', 'làm gì', 'quan hệ', 'đại học bách khoa hà nội', 'hai', 'của', 'làm gì', 'đại học khoa học tự nhiên', 'đóng', 'cha', 'làm gì', 'như vậy', 'quả', 'điều kiện', 'nè', 'tương lai', 'bác sĩ', 'học kỳ', 'đảng cộng sản việt nam', 'vấn đề', 'chúng ta', 'rồi', 'linh tinh', 'người yêu', 'điện thoại', 'suy nghĩ', 'nặng', 'học', 'dừng', 'hạnh phúc', 'hi ha', 'quan tâm', 'thích', 'chuyện', 'lạnh', 'phây', 'như thế này', 'lúc', 'hai', 'nghía', 'mới', 'hồ sơ', 'con trai', 'người yêu', 'đi', 'rồi', 'chị', 'kinh', 'kết bạn', 'hích', 'dễ thương', 'nhiều', 'chương trình', 'mình', 'mình', 'người', 'vợ chồng', 'ừm', 'thì', 'người yêu cũ', 'thanks', 'nàng', 'thôi', 'điên', 'bạn gái', 'với', 'xinh', 'hành động', 'đại học', 'mình', 'bạn', 'thích', 'chị', 'mọi người', 'người', 'nóng', 'không', 'không', 'biết', 'với', 'cũng', 'mình', 'what the fuck', 'được', 'comment', 'chồng', 'chồng', 'người ta', 'gia đình', 'ồ', 'vợ', 'công tác', 'sài gòn', 'anh em', 'à', 'ạ', 'gì', 'mới', 'việt nam', 'nha', 'cũng', 'đang', 'ơi', 'hích', 'anh chị em', 'à', 'ừ', 'con mẹ mày', 'con mẹ nó rồi', 'ơi', 'hôm nay', 'ừm', 'trung quốc', 'chương trình', 'đi', 'nói chuyện', 'triệu', 'ha ha', 'người ta', 'nghèo', 'kênh', 'à', 'admin', 'gì', 'người yêu', 'được', 'quảng cáo', 'bao giờ', 'vui', 'vẻ', 'tim', 'anh yêu em', 'em yêu anh', 'facebook', 'instagram', 'vậy', 'thích', 'vờ cờ lờ', 'điện thoại', 'account', 'lồn', 'lồn', 'lồn', 'trước', 'chẳng hiểu sao', 'đéo hiểu sao', 'quá', 'như thế nào', 'quá', 'vậy', 'vô', 'youtube', 'vãi đái', 'vãi chưởng', 'sấp mặt lờ', 'xin lỗi', 'con mẹ nó', 'facebook', 'hi hi', 'vui vẻ', 'nói', 'yêu', 'vãi cải đái', 'info', 'quằn què', 'subcribe', 'không', 'vậy', 'rồi', 'giờ', 'cái lồn mẹ mày', 'buổi sáng vui vẻ', 'trai', 'quá', 'hix', 'em', 'ý', 'gì', 'chị em', 'làm', 'đẹp giai', 'sorry', 'inbox', 'thôi', 'đéo bao giờ', 'không', 'ví dụ', 'anh', 'cũng vậy', 'vậy là', 'unfriend', 'my friend', 'friend', 'công ty', 'online', 'u23', 'không', 'đội tuyển quốc gia', 'hôm qua', 'xong', 'ừ', 'ừ', 'nhé', 'biết', 'quý', 'số tài khoản', 'hồng kông', 'được', 'ngành', 'nghĩa vụ quân sự', 'người', 'trong', 'thời gian', 'biết', 'phải', 'người', 'thế đéo nào', 'bình thường', 'vãi cả đái', 'thế giới di động', 'không', 'nhưng', 'trung học phổ thông', 'thằng', 'được', 'được', 'được', 'à', 'à', 'cu', 'thím', 'online', 'dô', 'vú', 'chứng minh nhân dân', 'số điện thoại', 'không liên quan']
    data = {"code" : code, "chuanhoa" : chuanhoa}
    teencode = pd.DataFrame(data)
    result = [x.strip() for x in sentence.split()]
    for i in range(0, len(result)):
        for j in range(0, len(teencode)):
            if (result[i] == teencode.at[j, "code"]):
                result[i] = teencode.at[j, "chuanhoa"]
    x = " ".join(result)
    x.strip()
    return x

In [ ]:
def remove_emoji(text):
    for emot in UNICODE_EMOJI:
        text = str(text).replace(emot, ' ')
    text = re.sub('  +', ' ', text).strip()
    return text
def url(text):
    text = re.sub(r'https?://\S+|www\.\S+', ' ', str(text))
    text = re.sub('  +', ' ', text).strip()
    return text
def mail(text):
    text = re.sub(r'[^@]+@[^@]+\.[^@]+', ' ', text)
    text = re.sub('  +', ' ', text).strip()
    return text
    # remove mention tag and hashtag
def tag(text):
    text = re.sub(r"(?:\@|\#|\://)\S+", " ", text)
    text = re.sub('  +', ' ', text).strip()
    return text

In [ ]:
c2e_path = '/content/drive/MyDrive/[1] Học tập/[2022 - 2023] Học kỳ 2/DS200 - DỮ LIỆU LỚN/Đồ án/CODE/Kafka - Spark - Model/Preprocess Data/character2emoji.xlsx'
character2emoji = pd.read_excel(c2e_path)  # character to emoji

In [ ]:
def convert_character2emoji(text):
    text = str(text)
    for i in range(character2emoji.shape[0]):
        text = text.replace(character2emoji.at[i, 'character'], " " + character2emoji.at[i, 'emoji'] + " ")
    text = re.sub('  +', ' ', text).strip()
    return text
e2w_path = '/content/drive/MyDrive/[1] Học tập/[2022 - 2023] Học kỳ 2/DS200 - DỮ LIỆU LỚN/Đồ án/CODE/Kafka - Spark - Model/Preprocess Data/emoji2word.xlsx'
emoji2word = pd.read_excel(e2w_path)

In [ ]:
def convert_emoji2word(text):
    for i in range(emoji2word.shape[0]):
        text = text.replace(emoji2word.at[i, 'emoji'], " " + emoji2word.at[i, 'word_vn'] + " ")
    text = re.sub('  +', ' ', text).strip()
    return text

In [ ]:
def special_character_1(text):  # remove dot and comma
    text = re.sub("[.,?!]", " ", text)
    text = re.sub('  +', ' ', text).strip()
    return text

In [ ]:
def abbreviation_kk(text):
    text = str(text)
    for t in text.split():
        if 'kk' in t:
            text = text.replace(t, ' ha ha ')
        else:
            if 'kaka' in t:
                text = text.replace(t, ' ha ha ')
            else:
                if 'kiki' in t:
                    text = text.replace(t, ' ha ha ')
                else:
                    if 'haha' in t:
                        text = text.replace(t, ' ha ha ')
                    else:
                        if 'hihi' in t:
                            text = text.replace(t, ' ha ha ')
    text = re.sub('  +', ' ', text).strip()
    return text

In [ ]:
def tokenized_VNCoreNLP(text):
    sentences = vncorenlp.tokenize(text)
    return " ".join(list(itertools.chain.from_iterable(sentences)))

In [ ]:
def text_preprocess_tokenize(document):
     # đưa về lower
     document = document.lower()
     # chuẩn hóa unicode
     document = covert_unicode(document)
     # xoá emoji
     document = deEmojify(document)
     # chuẩn hóa cách gõ dấu tiếng Việt
     document = chuan_hoa_dau_cau_tieng_viet(document)
     # xóa các ký tự không cần thiết
     document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)
     # chuẩn hoá teencode
     document = chuan_hoa_teen_code(document)
     # tách từ sử dụng VnCoreNLP
     document = tokenized_VNCoreNLP(document)
     # xóa khoảng trắng thừa
     document = re.sub(r'\s+', ' ', document).strip()

     document = convert_character2emoji(document)
     document = url(document)
     document = mail(document)
     document = tag(document)
     document = special_character_1(document)  # ##remove , . ? !
     document = abbreviation_kk(document)
     document = convert_character2emoji(document)
     document = remove_emoji(document)

     return document

In [ ]:
def text_preprocess_no_tokenize(document):
    # đưa về lower
    document = document.lower()
    # chuẩn hóa unicode
    document = covert_unicode(document)
    # xoá emoji
    document = deEmojify(document)
    # chuẩn hóa cách gõ dấu tiếng Việt
    document = chuan_hoa_dau_cau_tieng_viet(document)
    # xóa các ký tự không cần thiết
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)
    # chuẩn hoá teencode
    document = chuan_hoa_teen_code(document)
    # tách từ sử dụng VnCoreNLP
    #document = tokenized_VNCoreNLP(document)
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()

    document = convert_character2emoji(document)
    document = url(document)
    document = mail(document)
    document = tag(document)
    document = special_character_1(document)  # ##remove , . ? !
    document = abbreviation_kk(document)
    document = convert_character2emoji(document)
    document = remove_emoji(document)

    return document

In [ ]:
def preprocess_data_no_tokenize(data_full_label):
    # data_full_label = data_full_label.sample(len(data_full_label))
    data_full_label = data_full_label.dropna()
    for i in tqdm(range(len(data_full_label))):
        data_full_label.loc[i, "text"] = text_preprocess_no_tokenize(str(data_full_label.loc[i, "text"]))
    # data_full_label = data_full_label.sample(len(data_full_label))
    data_full_label = data_full_label.dropna()
    data_full_label.drop_duplicates().reset_index(drop=True)
    return data_full_label

# TRAIN MODEL

In [ ]:
vihsd_train = pd.read_csv('/content/drive/MyDrive/[1] Học tập/[2022 - 2023] Học kỳ 2/DS200 - DỮ LIỆU LỚN/Đồ án/DATA/Original Data/ViHSD/train.csv')

In [ ]:
!./kafka_2.13-3.4.1/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic vihsd-train

Created topic vihsd-train.


In [ ]:
X_train_dataset = vihsd_train.drop(["label"], axis=1)
y_train_dataset = vihsd_train["label"]

In [ ]:
X_train = list(filter(None, X_train_dataset.to_csv(index=False).split("\n")[1:]))
y_train = list(filter(None, y_train_dataset.to_csv(index=False).split("\n")[1:]))

In [ ]:
def error_callback(exc):
    raise Exception('Error while sendig data to kafka: {0}'.format(str(exc)))

In [ ]:
def write_to_kafka_train(topic_name, items):
  count=0
  producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])
  for message, key in items:
    producer.send(topic_name, key=key.encode('utf-8'), value=message.encode('utf-8')).add_errback(error_callback)
    count+=1
  producer.flush()
  print("Wrote {0} messages into topic: {1}".format(count, topic_name))

In [ ]:
write_to_kafka_train("vihsd-train", zip(X_train, y_train))

Wrote 8917 messages into topic: vihsd-train


In [ ]:
NUM_COLUMNS = len(X_train_dataset.columns)

In [ ]:
def decode_kafka_item(item):
    message = item.message  # Assuming the message is a scalar string
    key = tf.strings.to_number(item.key)
    return (message, key)

In [ ]:
BATCH_SIZE=64
SHUFFLE_BUFFER_SIZE=64
train_ds = tfio.IODataset.from_kafka('vihsd-train', partition=0, offset=0)
train_ds = train_ds.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)
train_ds = train_ds.map(decode_kafka_item)
train_ds = train_ds.batch(BATCH_SIZE)

In [ ]:
OPTIMIZER="adam"
LOSS=tf.keras.losses.BinaryCrossentropy(from_logits=True)
METRICS=['accuracy']
EPOCHS=10

In [ ]:
preprocessing_layer = tf.keras.layers.TextVectorization(
    max_tokens=10000,  # Specify the maximum number of tokens in your vocabulary
    output_mode='int',  # Output integer indices
    output_sequence_length=220  # Specify the desired sequence length
)
train_messages = train_ds.map(lambda x, y: x)  # Extract only the input data
preprocessing_layer.adapt(train_messages)

## Model Original

In [ ]:
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Embedding(preprocessing_layer.vocabulary_size(), 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.build(input_shape=(NUM_COLUMNS,))
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 220)              0         
 ectorization)                                                   
                                                                 
 embedding_3 (Embedding)     (None, 220, 64)           640000    
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              66048     
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 714,369
Trainable params: 714,369
Non-tr

In [ ]:
model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)

In [ ]:
model.fit(train_ds, epochs=EPOCHS)

Epoch 1/10
140/140 [==============================] - 47s 277ms/step - loss: 0.5551 - accuracy: 0.7045
Epoch 2/10
140/140 [==============================] - 16s 112ms/step - loss: 0.2993 - accuracy: 0.8730
Epoch 3/10
140/140 [==============================] - 17s 118ms/step - loss: 0.1892 - accuracy: 0.9317
Epoch 4/10
140/140 [==============================] - 10s 73ms/step - loss: 0.1279 - accuracy: 0.9584
Epoch 5/10
140/140 [==============================] - 9s 64ms/step - loss: 0.1011 - accuracy: 0.9673
Epoch 6/10
140/140 [==============================] - 7s 48ms/step - loss: 0.0848 - accuracy: 0.9729
Epoch 7/10
140/140 [==============================] - 11s 78ms/step - loss: 0.0632 - accuracy: 0.9809
Epoch 8/10
140/140 [==============================] - 7s 47ms/step - loss: 0.0669 - accuracy: 0.9786
Epoch 9/10
140/140 [==============================] - 8s 52ms/step - loss: 0.0617 - accuracy: 0.9805
Epoch 10/10
140/140 [==============================] - 9s 60ms/step - loss: 0.0463 

## Model Enhanced

In [ ]:
# Load the XLM-RoBERTa model and tokenizer
# tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
# model = XLMRobertaModel.from_pretrained('xlm-roberta-base')

# Define the XLM-RoBERTa model
model_enhance = tf.keras.Sequential([
  preprocessing_layer,
  # Embedding layer
  tf.keras.layers.Embedding(preprocessing_layer.vocabulary_size(), 64),
  # Bidirectional GRU layer
  tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True)),
  # Conv1D layer
  tf.keras.layers.Conv1D(64, (3,), activation='relu'),
  # MaxPooling1D layer
  tf.keras.layers.MaxPooling1D(pool_size=(2,)),
  # Flatten layer
  tf.keras.layers.Flatten(),
  # Dense layer
  tf.keras.layers.Dense(128, activation='relu'),
  # Dense layer with sigmoid activation for classification
  tf.keras.layers.Dense(1, activation='sigmoid')
])

# Set the input shape
model_enhance.build(input_shape=(NUM_COLUMNS,))

# Print the model summary
print(model_enhance.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 220)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 220, 64)           640000    
                                                                 
 bidirectional (Bidirectiona  (None, 220, 256)         148992    
 l)                                                              
                                                                 
 conv1d (Conv1D)             (None, 218, 64)           49216     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 109, 64)          0         
 )                                                               
                                                        

In [ ]:
model_enhance.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)

In [ ]:
model_enhance.fit(train_ds, epochs=EPOCHS)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5703: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


140/140 [==============================] - 58s 283ms/step - loss: 0.5519 - accuracy: 0.7083
Epoch 2/10
140/140 [==============================] - 15s 104ms/step - loss: 0.3082 - accuracy: 0.8748
Epoch 3/10
140/140 [==============================] - 11s 77ms/step - loss: 0.1848 - accuracy: 0.9323
Epoch 4/10
140/140 [==============================] - 8s 58ms/step - loss: 0.1241 - accuracy: 0.9564
Epoch 5/10
140/140 [==============================] - 8s 58ms/step - loss: 0.0925 - accuracy: 0.9686
Epoch 6/10
140/140 [==============================] - 9s 60ms/step - loss: 0.0769 - accuracy: 0.9739
Epoch 7/10
140/140 [==============================] - 8s 58ms/step - loss: 0.0693 - accuracy: 0.9749
Epoch 8/10
140/140 [==============================] - 11s 78ms/step - loss: 0.0433 - accuracy: 0.9864
Epoch 9/10
140/140 [==============================] - 8s 54ms/step - loss: 0.0385 - accuracy: 0.9886
Epoch 10/10
140/140 [==============================] - 6s 43ms/step - loss: 0.0308 - accuracy: 0

# **PIPELINE**

In [ ]:
def create_topic(topic_name):
    !./kafka_2.13-3.4.1/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic "{topic_name}"

In [ ]:
def error_callback(exc):
    raise Exception('Error while sendig data to kafka: {0}'.format(str(exc)))

In [ ]:
def write_to_kafka(topic_name, items):
    count = 0
    producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])
    for message in items:  # Removed 'key' since it's not used
        producer.send(topic_name, value=message.encode('utf-8')).add_errback(error_callback)
        count += 1
    producer.flush()
    print("Wrote {0} messages into topic: {1}".format(count, topic_name))

In [ ]:
def decode_kafka_test_item(raw_message, raw_key):
    message = raw_message
    key = tf.strings.to_number(raw_key)
    return message

In [ ]:
# Đường dẫn đến thư mục chứa các file csv
folder_path = '/content/drive/MyDrive/[1] Học tập/[2022 - 2023] Học kỳ 2/DS200 - DỮ LIỆU LỚN/Đồ án/DATA/Real-time Data/Youtube Live Chat/'

In [ ]:
# Hàm kiểm tra xem có file mới trong thư mục hay không
def check_for_new_files(last_index):
    files = os.listdir(folder_path)
    # Tìm các file có đuôi .csv
    csv_files = [file for file in files if file.endswith('.csv')]
    # Lọc ra những file mới hơn last_index
    new_files = [file for file in csv_files if int(file.split('_')[-1].split('.')[0]) > last_index]
    if new_files:
        return new_files # Trả về danh sách các file mới
    return None

In [ ]:
# Lấy số index lớn nhất trong danh sách file
def get_max_index(files):
    max_index = -1
    for file in files:
        index = int(file.split('_')[-1].split('.')[0])
        if index > max_index:
            max_index = index
    return max_index

In [ ]:
BATCH_SIZE = 64
# Vòng lặp chờ đợi và đọc file mới
last_index = 0  # Chưa có file nào được đọc
topic_name = "streaming_real_time"
create_topic(topic_name)
while True:
    new_files = check_for_new_files(last_index)
    if new_files:
        for new_file in new_files:
            file_path = os.path.join(folder_path, new_file)

            data_real_time = pd.read_csv(file_path)

            data_real_time = preprocess_data_no_tokenize(data_real_time)

            data_real_time = list(filter(None, data_real_time.to_csv(index=False).split("\n")[1:]))

            write_to_kafka(topic_name, data_real_time)

            data_stream = tfio.experimental.streaming.KafkaGroupIODataset(
                          topics=[topic_name],
                          group_id="clgt",
                          servers="127.0.0.1:9092",
                          stream_timeout=10000,
                          configuration=[
                              "session.timeout.ms=7000",
                              "max.poll.interval.ms=8000",
                              "auto.offset.reset=earliest"
                          ],)

            data_stream = data_stream.map(decode_kafka_test_item)
            data_stream = data_stream.batch(BATCH_SIZE)

            iterator = data_stream.__iter__()
            all_predictions = []
            for batch in iterator:
                sentences = batch.numpy()
                predictions = model_enhance.predict(batch)
                for i, sentence in enumerate(sentences):
                    sentence_text = sentence.decode('utf-8')
                    prediction = predictions[i]
                    label = 0 if prediction >= 0.5 else 1
                    print(f"Sentence: {sentence_text}  -  Prediction: {prediction}  -  Label: {'Hate' if label == 0 else 'Clean'}")
                    all_predictions.append((sentence_text, prediction, label))

             # Cập nhật last_index
            last_index = max(last_index, int(new_file.split('_')[-1].split('.')[0]))
    else:
        # Nếu không có file mới, chờ 1 khoảng thời gian trước khi kiểm tra lại
        time.sleep(5)  # Chờ 5 giây trước khi kiểm tra lại

Error while executing topic command : Topic 'streaming_real_time' already exists.
[2023-07-21 07:45:06,290] ERROR org.apache.kafka.common.errors.TopicExistsException: Topic 'streaming_real_time' already exists.
 (kafka.admin.TopicCommand$)


100%|██████████| 10/10 [00:00<00:00, 53.28it/s]


Wrote 10 messages into topic: streaming_real_time
1/1 [==============================] - 0s 35ms/step
Sentence: kiếm mấy con 8 113 á anh  -  Prediction: [0.01501325]  -  Label: Clean
Sentence: anh tìm 8 ebs hay cap có mấy con 20 tỷ ấy anh  -  Prediction: [3.5154949e-06]  -  Label: Clean
Sentence: mùa 22u với hg ấy anh  -  Prediction: [7.2602306e-05]  -  Label: Clean
Sentence: 22u 3 anh oiw  -  Prediction: [0.02108111]  -  Label: Clean
Sentence: thank you for subscribing shyro  -  Prediction: [2.6056638e-05]  -  Label: Clean
Sentence: 22u 3 đầy anh  -  Prediction: [3.864909e-07]  -  Label: Clean
Sentence: m4 horus hôm qua như thế nào rồi anh hôm qua em không xem được hết stream  -  Prediction: [0.00691939]  -  Label: Clean
Sentence: longconga  -  Prediction: [0.14310214]  -  Label: Clean
Sentence: lông con gà  -  Prediction: [0.79873955]  -  Label: Hate
Sentence: fifa phải trải nghiệm nhiều team mới sướng anh nhỉ  -  Prediction: [9.1184954e-10]  -  Label: Clean


100%|██████████| 10/10 [00:00<00:00, 123.35it/s]


Wrote 10 messages into topic: streaming_real_time
1/1 [==============================] - 0s 23ms/step
Sentence: ""  -  Prediction: [0.11401407]  -  Label: Clean
Sentence: heloo  -  Prediction: [0.00152344]  -  Label: Clean
Sentence: lươn  -  Prediction: [0.14310214]  -  Label: Clean
Sentence: hí anhhhhh  -  Prediction: [0.00082555]  -  Label: Clean
Sentence: 50 cai dau anh  -  Prediction: [0.99806064]  -  Label: Hate
Sentence: diễn 10 điểm  -  Prediction: [0.04994104]  -  Label: Clean
Sentence: hello chú độp  -  Prediction: [2.9726416e-05]  -  Label: Clean
Sentence: hít đất  -  Prediction: [0.59698546]  -  Label: Hate
Sentence: 50 cái  -  Prediction: [0.6584181]  -  Label: Hate
Sentence: anh phét  -  Prediction: [0.93217146]  -  Label: Hate


...

Wrote 10 messages into topic: streaming_real_time
1/1 [==============================] - 0s 24ms/step
Sentence: hit dat 50 cai  -  Prediction: [0.9673612]  -  Label: Hate
Sentence: 50 cái chú ơi  -  Prediction: [0.00525967]  -  Label: Clean
Sentence: vâng  -  Prediction: [0.13018437]  -  Label: Clean
Sentence: hí lô  -  Prediction: [0.00048755]  -  Label: Clean
Sentence: 50 cái hít đất  -  Prediction: [0.9423183]  -  Label: Hate
Sentence: mmmm  -  Prediction: [0.14310214]  -  Label: Clean
Sentence: khiếp  -  Prediction: [0.00322906]  -  Label: Clean
Sentence: hít đất 50c  -  Prediction: [0.6004923]  -  Label: Hate
Sentence: ""  -  Prediction: [0.11401407]  -  Label: Clean
Sentence: nua nua  -  Prediction: [0.99587303]  -  Label: Hate


100%|██████████| 10/10 [00:00<00:00, 145.13it/s]


Wrote 10 messages into topic: streaming_real_time
1/1 [==============================] - 0s 24ms/step
Sentence: 50 cái chú ơiiiii  -  Prediction: [8.37572e-07]  -  Label: Clean
Sentence: 50 hít đất  -  Prediction: [0.77665395]  -  Label: Hate
Sentence: hứa thanh độ  -  Prediction: [0.00122945]  -  Label: Clean
Sentence: 50 cái hít đát  -  Prediction: [0.95671594]  -  Label: Hate
Sentence: hí anh độp  -  Prediction: [0.00038643]  -  Label: Clean
Sentence: 50 cái hít đất  -  Prediction: [0.9423183]  -  Label: Hate
Sentence: 50 cái hít đất đi anh  -  Prediction: [0.8952705]  -  Label: Hate
Sentence: hí chú độpp aạaa  -  Prediction: [0.00030594]  -  Label: Clean
Sentence: chào anh mixi  -  Prediction: [0.00224006]  -  Label: Clean
Sentence: đây gòi  -  Prediction: [0.35419255]  -  Label: Clean


100%|██████████| 10/10 [00:00<00:00, 165.24it/s]


Wrote 10 messages into topic: streaming_real_time
